In [12]:
import glob
import matplotlib.pyplot as plt
import cv2
import numpy as np
import torch
import torch.nn as nn
import tqdm

from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam
from torch.utils.data.dataset import Dataset
from PIL import Image

In [25]:
## RGB를 LAB로 변환하는 함수
def rgb2lab(rgb):
    return cv2.cvtColor(rgb, cv2.COLOR_RGB2LAB)

## LAB를 RGB로 변환하는 함수
def lab2rgb(lab):
    return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

## 데이터셋 정의 클래스
class AutoColoring(Dataset):
    def __init__(self):
        self.data = glob.glob("./data/archive/img_align_celeba/img_align_celeba/*.jpg")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        rgb = np.array(Image.open(self.data[i]).resize((256, 256)))
        lab = rgb2lab(rgb)
        lab = lab.transpose((2, 0, 1)).astype(np.float32) # 파이토치는 채널이 가장 앞에 와야 하므로 transpose를 사용했습니다.

        return lab[0], lab[1:]

In [26]:
## Low level 특징 추출하는 클래스
class LowLevel(nn.Module):
    def __init__(self):
        super(LowLevel, self).__init__()

        self.low1 = nn.Conv2d(1, 64, kernel_size=3, stride=2, padding=1)
        self.lb1 = nn.BatchNorm2d(64)
        self.low2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.lb2 = nn.BatchNorm2d(128)
        self.low3 = nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1)
        self.lb3 = nn.BatchNorm2d(128)
        self.low4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.lb4 = nn.BatchNorm2d(256)
        self.low5 = nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1)
        self.lb5 = nn.BatchNorm2d(256)
        self.low6 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.lb6 = nn.BatchNorm2d(512)

        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        low = self.low1(x)
        low = self.lb1(low)
        low = self.sigmoid(low)

        low = self.low2(low)
        low = self.lb2(low)
        low = self.sigmoid(low)

        low = self.low3(low)
        low = self.lb3(low)
        low = self.sigmoid(low)

        low = self.low4(low)
        low = self.lb4(low)
        low = self.sigmoid(low)

        low = self.low5(low)
        low = self.lb5(low)
        low = self.sigmoid(low)

        low = self.low6(low)
        low = self.lb6(low)
        low = self.sigmoid(low)

        return low
    
## Middle level 특징 추출하는 클래스
class MidLevel(nn.Module):
    def __init__(self):
        super(MidLevel, self).__init__()

        self.mid1 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.mb1 = nn.BatchNorm2d(512)
        self.mid2 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1)
        self.mb2 = nn.BatchNorm2d(256)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        mid = self.mid1(x)
        mid = self.mb1(mid)
        mid = self.sigmoid(mid)

        mid = self.mid2(mid)
        mid = self.mb2(mid)
        mid = self.sigmoid(mid)

        return mid

## Global level 특징 추출하는 클래스
class GlobalLevel(nn.Module):
    def __init__(self):
        super(GlobalLevel, self).__init__()

        self.glob1 = nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1)
        self.gb1 = nn.BatchNorm2d(512)
        self.glob2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.gb2 = nn.BatchNorm2d(512)
        self.glob3 = nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1)
        self.gb3 = nn.BatchNorm2d(512)
        self.glob4 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.gb4 = nn.BatchNorm2d(512)

        # 글로벌 레벨 특징 추출기의 MLP층 구성하여 색을 칠하는 특징으로 사용합니다.
        self.fc1 = nn.Linear(in_features=32768, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=512)
        self.fc3 = nn.Linear(in_features=512, out_features=256)

        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        glo = self.glob1(x)
        glo = self.gb1(glo)
        glo = self.sigmoid(glo)

        glo = self.glob2(glo)
        glo = self.gb2(glo)
        glo = self.sigmoid(glo)

        glo = self.glob3(glo)
        glo = self.gb3(glo)
        glo = self.sigmoid(glo)

        glo = self.glob4(glo)
        glo = self.gb4(glo)
        glo = self.sigmoid(glo)

        # 추출된 특징을 1차원으로 펼쳐줍니다.
        glo = torch.flatten(glo, start_dim=1)
        glo = self.fc1(glo)
        glo = self.sigmoid(glo)
        glo = self.fc2(glo)
        glo = self.sigmoid(glo)
        glo = self.fc3(glo)
        glo = self.sigmoid(glo)

        return glo
    
## Colorization Network 클래스
class Colorization(nn.Module):
    def __init__(self):
        super(Colorization, self).__init__()
        
        self.color1 = nn.ConvTranspose2d(256, 128, 3, 1, 1)
        self.cb1 = nn.BatchNorm2d(128)
        self.color2 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.cb2 = nn.BatchNorm2d(64)
        self.color3 = nn.ConvTranspose2d(64, 64, 3, 1, 1)
        self.cb3 = nn.BatchNorm2d(64)
        self.color4 = nn.ConvTranspose2d(64, 32, 2, 2)
        self.cb4 = nn.BatchNorm2d(32)
        self.color5 = nn.ConvTranspose2d(32, 2, 2, 2)

        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        color = self.color1(x)
        color = self.cb1(color)
        color = self.sigmoid(color)
        color = self.color2(color)
        color = self.cb2(color)
        color = self.sigmoid(color)
        color = self.color3(color)
        color = self.cb3(color)
        color = self.sigmoid(color)
        color = self.color4(color)
        color = self.cb4(color)
        color = self.sigmoid(color)
        color = self.color5(color)

        return color

## AutoColoring 모델 정의
class AutoColoringModel(nn.Module):
    def __init__(self):
        super(AutoColoringModel, self).__init__()

        self.low = LowLevel()
        self.mid = MidLevel()
        self.glob = GlobalLevel()
        self.fusion = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1) # 특징 합치기 위한 것입니다.
        self.color = Colorization()
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        low = self.low(x)
        mid = self.mid(low)
        glo = self.glob(low)
        
        # 글로벌 레벨 특징 추출기의 출력을 미들 레벨 특징 추출기의 출력 크기가 되도록 반복합니다.
        fusion = glo.repeat(1, mid.shape[2]*mid.shape[2]) 
        fusion = torch.reshape(fusion, (-1, 256, mid.shape[2], mid.shape[2]))
        
        # 글로벌 레벨 특징 추출기의 특징과 미들 레벨 특징 추출기의 특징을 결합합니다.
        fusion = torch.cat([mid, fusion], dim=1)
        fusion = self.fusion(fusion)
        fusion = self.sigmoid(fusion)

        color = self.color(fusion)

        return color

In [27]:
## 모델 정의와 데이터 정의
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoColoringModel().to(DEVICE)
dataset = AutoColoring()
loader = DataLoader(dataset, batch_size=32, shuffle=True)
optim = Adam(params=model.parameters(), lr=0.01)

## 학습
for epoch in range(201):
    print(f'Epoch {epoch + 1} of {200}')
    
    iterator = tqdm.tqdm(loader)
    for L, AB in iterator:        
        L = torch.unsqueeze(L, dim=1).to(DEVICE) # L 채널은 흑백 이미지 이므로 채널 차원을 확보했습니다.
        optim.zero_grad()

        pred = model(L)
        loss = nn.MSELoss()(pred, AB.to(DEVICE))
        loss.backward()
        optim.step()
        
        iterator.set_description(f"epoch:{epoch} loss:{loss.item()}")

torch.save(model.state_dict(), "AutoColor.pth")

Epoch 1 of 200



epoch:0 loss:13174.931640625:   1%|▎                    | 83/6332 [18:15<22:49:00, 13.14s/it]


epoch:0 loss:7568.6875:   3%|▋                         | 167/6332 [36:14<21:46:31, 12.72s/it]


epoch:0 loss:4003.468505859375:   4%|▋                 | 251/6332 [54:36<21:16:45, 12.60s/it]


epoch:0 loss:2026.809814453125:   5%|▊               | 335/6332 [1:12:34<19:00:54, 11.41s/it]


...

epoch:0 loss:96.48887634277344:  49%|██████▊       | 3107/6332 [27:19:10<13:08:52, 14.68s/it]


epoch:0 loss:137.31442260742188:  50%|██████▌      | 3191/6332 [27:37:57<12:35:02, 14.42s/it]


epoch:0 loss:135.328125:  52%|██████████▊          | 3275/6332 [30:40:59<83:54:09, 98.81s/it]


epoch:0 loss:128.6634521484375:  53%|███████▍      | 3340/6332 [43:53:29<39:19:06, 47.31s/it]


KeyboardInterrupt: 

In [ ]:
# pyplot의 이미지 형식에 맞추기 위한 약간의 변형합니다.
test_L, test_AB = dataset[0]
test_L = np.expand_dims(test_L, axis=0)
real_img = np.concatenate([test_L, test_AB])
real_img = real_img.transpose(1, 2, 0).astype(np.uint8)
real_img = lab2rgb(real_img)

with torch.no_grad():
    model.load_state_dict(torch.load("AutoColor.pth", map_location=device))

    input_tensor = torch.tensor(test_L)
    input_tensor = torch.unsqueeze(input_tensor, dim=0).to(device)
    pred_AB = model(input_tensor)
   
    pred_LAB = torch.cat([input_tensor, pred_AB], dim=1)
    pred_LAB = torch.squeeze(pred_LAB)
    pred_LAB = pred_LAB.permute(1, 2, 0).cpu().numpy()
    pred_LAB = lab2rgb(pred_LAB.astype(np.uint8))

plt.subplot(1, 2, 1)
plt.imshow(real_img)
plt.title("real image")
plt.subplot(1, 2, 2)
plt.imshow(pred_LAB)
plt.title("predicted image")
plt.show()